# Retention factor (Rf) value calculator for TLC plates

#### How to use the programme: 
Upload an image of your TLC plate by entering the file name of the image of your TLC plate, showing the solvent front and baseline with the spots circled in pencil/pen. The Rf values of the spots will automatically be calculated and shown in the programme. An image of the analysed plate, showing the Rf values of each spot will be save in the same file as the orginal input image.

##### Note: The programme only works for UV-visualised TLC plates and TLC plates with coloured spots. NOT stain-visualised plates. 

#### How the Rf value is calculated:

$$ R_{f}\ {value} = \frac{Distance\ travelled\ by\ the\ compound}{Distance\ travelled\ by\ the\ solvent\ front} $$

#### Image requirements:
* Accepted image formats: **.jpg**, **.jpeg**, **.png**, **.tiff**, and **.tif**.
* Image file size must be between 150 kB to 10 MB.
* The TLC plate must be on a dark contrasting background for programme to work well.

#### Example of a good image input:

<img src ="display_image2_after.jpg" alt="Drawing" style="width: 250px;"/>

*To make spot detection easier and more accurate, circle around the oval spot outlines, as shown in example.*

#### Example of the output image:

<img src ="display_image2_after_analysed.jpg" alt="Drawing" style="width: 200px;"/>

The image of the analysed plate will be displayed and saved in the same file.


In [ ]:
##LIBRARY IMPORT##
import cv2 #Imports opencv library for image processing
import numpy as np #Imports numpy
import os #Used to read files
from matplotlib import pyplot as plt #Imports pyplot to display images in the notebook
%matplotlib notebook
print("Libraries Imported")


In [ ]:
img_format = ["jpg","jpeg","png","tiff","tif"] #Allowed image formats

In [ ]:
##FUNCTIONS##
def input_check(filename):
    """
    This function checks the input file and the size of the file
    Checks to see if the file exist, if it is an accepted image file format and the file size is too big or too small
    Checks if image is greater than 150kB then resizes it to 30% of original size for easier handling
    If it is less than 500kB, the image is not resized.
    Returns img_resized
    """
    #Error message for if the filename does not exist
    assert (os.path.isfile(filename)),"File does not exist.\nNo file has been uploaded. Please try again."
    
    #If the file exist, split at "." to check if the filetype is an accepted image format
    file_check = filename.split(".")
    #Error message to check for extra fullstops in the filename
    assert (len(file_check) == 2), "Please rename file to continue. A fullstop'.' in the file name is forbidden"
    file_type = file_check[1]
    
    #Error message for if  the filetype is not in the img_format list
    assert (file_type in img_format), "File uploaded is not an accepted image format.\nPlease try again."
    
    file_size = os.path.getsize(filename) #Gets filesize in bytes
    filesize_kb = file_size * (10**-3) #Converts bytes to kB
    filesize_mb = file_size * (10**-6)
    img = cv2.imread(filename) #Reads image using OpenCV
    dimensions = img.shape
    
    #Error message for if the image filesize is greater than 10 MB or smaller than 150kb
    assert (filesize_mb < 10), "Image uploaded is too large. File size limit is 10 MB.\nPlease try again."
    assert (filesize_kb > 150), "Image uploaded is too small. The image quality will not be good enough for Rf calculations to take place.\nPlease try again."
    
    #Resizing image for easier handling
    if filesize_kb > 800 and filesize_mb <= 2 :  #If the image size is greater that 800kb and less than 2 MB
        scale_factor = 0.6 #Rescales image to 60% of original size
    elif filesize_mb > 2: #If image is greater than 2 MB
        scale_factor = 0.3 #Rescales image to 30% of original size
    else: #If the image is less than 800kb and greater than 150kb
        scale_factor = 1 #No change in size
    #print(scale_factor)
    width = int( dimensions[1] * scale_factor )
    height = int( dimensions[0] * scale_factor )
    img_resized = cv2.resize(img, (width, height))

    return img_resized

In [ ]:
#Image processing before calculating distances for TLC
def img_Processing(img):
    """
    This function processes the image file, img
    Img is turned to greyscale, Canny edge detection happens, closing function used (which combines dilation and erosion)
    Returns the processed image, img_processed
    """
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Image converted to grayscale image
    imgCanny = cv2.Canny(imgGray, 50, 50) #Canny edge detection
    kernel = np.ones((15, 15))
    img_processed = cv2.morphologyEx(imgCanny, cv2.MORPH_CLOSE, kernel) #Uses kernel to use closing function
    
    return img_processed

In [ ]:
#Contours function - to find the use biggest contour to identify the plate
def getcontours(img_processed):
    """
    This function find the contours of the img_processed from img_Processing function
    Identifies the contours/outline of the TLC plate
    Using the contours, the approximate 4 corners of the plate are found
    Returns corners, which is a 3 dimensional matrix with the coordinate of the plate
    """
    #img_Contour, copy of original image to check contours against
    #img_Contour = img.copy() 
    
    corners = np.array([])
    maxArea = 0
    #Retrieves the all the contours of the img_processed
    contours, hierarchy = cv2.findContours(img_processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for cont in contours: #Loops through all the points of the contours that define the contours of the image
        area = cv2.contourArea(cont) #Find the area of the region bounded by contours
        #print(area)
        if area > 8000: #Want the biggest area contoured, assuming it will be the plate
            # To check contours
            #img_Contour = cv2.drawContours(img_Contour, cont, -1, (0, 255, 0), 3) #Outlines the whole plate on imgContour
            edges = cv2.arcLength(cont, True) #Calculates length of edges of contour
            #print (edges)
            #Finds the approximate corners of the plate by using the edges defined for a closed contour area
            approx = cv2.approxPolyDP(cont, 0.015 * edges, True) #1.5% accuracy approximation for corners
            #To make sure there is a contoured area and 4 corners for the contoured area
            if area > maxArea and len(approx) == 4: 
                #Renaming varibles
                corners = approx
                maxArea = area
    #To display the corners on the img_Contour
    #img_Contour = cv2.drawContours(img_Contour, corners, -1, (255, 0, 0), 15)
    
    return corners #img_Contour #Change to imgContour to see original image with contours and the 4 corners

In [ ]:
def getdimensions(corners):
    """
    This function finds the dimensions of the plate using corners, to use for warping so the aspect ratio does not change.
    Uses the largest difference between the x coordinates of the corners as the new width
    Uses the largest difference between the y coordinates of the corners as the new height
    Append the new width first and then the new height to a list
    Returns dimensions, a list with [width, height]
    """
    #Error message for when there are not 4 corners detect and so warping cannot happen
    assert(len(corners) == 4), "No plate has been detected, so the image cannot be warped.\nPlease re-upload a better quality image."
    #Corners is 3 dimensional matrix, so reshape to 2 dimensional for easier handling
    points = corners.reshape((4,2))
    
    #Empty lists
    width_list = []
    height_list = []
    dimensions = [] #empty list for [width, height]
    
    for i in range(0, len(points) - 1 ): #For every row of the matrix/every point which are the corners of the plate
        #Find the difference between the x coordinates, which will give the width of the plate
        w_diff = int(points[i+1][0]) - int(points[i][0])
        #Append to width_list
        width_list.append(w_diff)
        #Find the difference between the y coordinates, which will give the height of the plate
        h_diff = int(points[i+1][1]) - int(points[i][1])
        #Append to width_list
        height_list.append(h_diff)
    
    #Let the width and height be the largest value of the lists
    width = (max(width_list))
    height = (max(height_list))
    
    #Append to dimensions as integers in the order [width, height]
    dimensions.append(int(width))
    dimensions.append(int(height))  
    
    return dimensions  #[width, height]

In [ ]:
def warp(img, corners):
    """
    This function reorders the corners found from getcontours function to the same order as defined in this function
    Defined order of points/ Newpoints: (Top left(0,0), Top right(x,0), Bottom right(0,y),Bottom left(x,y))
    First reorders the 4 points of the plate then warps image using the dimensions from getdimensions function
    Returns warped_img, flatten out image of the plate
    """
    dimensions= getdimensions(corners)
    while True:
    #print(dimesions)
    
        #Corners is 3 dimensional matrix, so reshape to 2 dimensional for easier handling
        corners = corners.reshape((4,2))
        #Create a new matrix to fill in with the points in correct order
        new_matrix = np.zeros ((4,1,2), np.int32)
    
        #Defined points order: Top left(0,0), Top right(x,0), Bottom right(0,y),Bottom left(x,y)
    
        #Use sum and differences to differentiate the points
        sum = np.sum(corners, axis=1)
        #Top left (0,0) will have smallest sum of all points and put point coordinates in the 1st row of new_matrix 
        new_matrix[0] = corners[np.argmin(sum)]
        #Bottom left(x,y) will have largest sum of all points and put point coordinates in the 4th row of new_matrix 
        new_matrix[3] = corners[np.argmax(sum)]
    
        diff = np.diff(corners,axis=1)
        #Top right (0,y) will have smallest difference and put point coordinates in the 2nd row of new_matrix 
        new_matrix[1] = corners[np.argmin(diff)]
        #Bottom right(x,0) will have largest difference and put point coordinates in the 3rd row of new_matrix 
        new_matrix[2] = corners[np.argmax(diff)]
        break
        #Rename new_matrix to points
    corners = new_matrix
    #print ('Points reordered')
    
    points_of_plate = np.float32(corners) #New ordered points matrix 
    #Newpoints defines the coordinated of where the corner points of the plate will be mapped to
    Newpoints = np.float32([[0, 0], [dimensions[0], 0], [0, dimensions[1]], [dimensions[0], dimensions[1]]])
    #Define order of points (Top left, Top right, Bottom right, Bottom left)
    #Uses the width and height from getdimensions to keep the same aspect ratio of the plate  
    
    #Transformation matrix to map points_of plate to Newpoints
    matrix = cv2.getPerspectiveTransform(points_of_plate, Newpoints)
    #Apply warp perspective to get the flat out image, keeping same aspect ratio as original plate
    warped_img = cv2.warpPerspective(img, matrix, (dimensions[0], dimensions[1]))
    
    return warped_img

In [ ]:
#Hough transform to find pencil line and solvent front
def all_lines(processed_plate):
    """
    This function detects all the lines in the image of the plate
    A copy of the plate is made, and lines are drawn on the copy to show
    Return lines, which is a matrix of the start and finish points of detected lines
    """
    #Copying and converting from gray to BGR so lines can be displayed on plate image
    copy = cv2.cvtColor(processed_plate, cv2.COLOR_GRAY2BGR)
   
    #Probabilistic Hough line transform to detect the major lines from plate,so it doesn't mistake long ovals as lines
    lines = cv2.HoughLinesP(processed_plate, 1, np.pi / 180, 100, 100, 100)
    
    #Error message if no lines are detected at all
    assert (lines is not None), "No lines detected on image of plate. Please re-upload a better quality image."
    
    #If there is detected lines, draws the all lines on copy of plate 
    if lines is not None: 
        for i in range(0, len(lines)):
            l = lines[i][0]
            #Draw lines in red
            cv2.line(copy, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv2.LINE_8)
            
    return lines #Change to copy to see line detection image

In [ ]:
#find distances between the solvent front and pencil line.
def distances(lines, length_threshold): 
    """
    This function finds the categorise each line using a y coordinate threshold
    The line will either be a detected solvent front line or baseline
    Then the average of all the y coordinates of the solvent front lines and baselines are found
    Finds distance between solvent front and baseline
    Returns a list with the average y coordinate for solvent front and baseline and the distance between them
    """
    
    #Defining variables needed for calculating the averages
    sf_count=0
    bl_count=0
    sf_ytotal=0
    bl_ytotal=0
    
    for r in range(0, len(lines)): #For each line/ each row in the line matrix
        #Check for each ycoordinate of the start point of the line that is not 0 
        if lines[r][1] != 0: 
            if lines[r][1] < length_threshold:# If y coord is less than threshold: belongs to SF
                y_1 = lines[r][1]#Let the y coordinate equal y_1 
                sf_ytotal = sf_ytotal + y_1 #Add y value to total
                sf_count = sf_count + 1 #Counter for each y value added to caluclate average for SF
            else: #Else the y coordinates belong to BL
                y_3 = lines[r][1] 
                bl_ytotal = bl_ytotal + y_3 #Add y value to total
                bl_count = bl_count + 1 #Counter for each y value added to caluclate average for BL
                
        #Repeat the same for y coordinate of the end point
        #Check for each y coordinate of the end point of the line that is not 0
        if lines[r][3] != 0:
            if lines[r][3] < length_threshold:
                y_2 = lines[r][3]
                sf_ytotal = sf_ytotal + y_2
                sf_count = sf_count + 1
            else:
                y_4 = lines[r][3]
                bl_ytotal = bl_ytotal + y_4
                bl_count = bl_count + 1
                
    #print(sf_count)
    #print(bl_count)
    
    assert(sf_count != 0),"No solvent front detected. Please re-upload a better quality image."
    assert(bl_count != 0),"No baseline detected. Please re-upload a better quality image."
    
    #Calculating averages    
    avg_bl = bl_ytotal / bl_count
    avg_sf = sf_ytotal / sf_count
    #print("The average y coord for SF", int( avg_sf ))
    #print("The average y coord for BL", int( avg_bl ))
    
    #Distance  between the solvent front and baseline in pixels  
    dist_sf_bl = int( avg_bl - avg_sf )    
    print ("Distance between solvent front and pencil line is", dist_sf_bl , "pixels")
    
    return [avg_sf, avg_bl, dist_sf_bl] 

In [ ]:
def findspots(plate):
    """
    This function uses hough circle transform to detect the spots on plate
    Uses cropped plate image, before img_processed, to convert to grayscale
    Returns circles, Matrix showing (x coord of centre,y coord of centre, radius) of all detected circles
    """
    grey = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY) #Converts cropped imagto greyscale
    grey = cv2.blur(grey,(5,5)) #Blurs image to prevent false circle detection
    
    #Hough circle transform to detect circles
    rows = grey.shape[0]
    circles = cv2.HoughCircles(grey, cv2.HOUGH_GRADIENT, 1, rows / 8, 
                           param1=40, param2=30,
                           minRadius=1,maxRadius=0)
    #print(circles) 
    assert(circles is not None),"No spots have been detected on plate. Please re-upload a better quality image."
    
    return circles

In [ ]:
def Rf(circles, positions, cropped_plate):
    """
    This function finds the centre of the circle/spots, and finds the distance of them from baseline
    Finds Rf by dividing the distance between spots and BL by the distance between SF and BL 
    Returns all_Rf, a list with all Rf values
    """
    #Defining variable required
    circle_num = 0 #Varible used for numbering the spots
    dist_sf_spot = 0 #Distance between centre of spot from solvent front (SF)
    dist_bl_spot =0 #Distance between centre of spot to baseline ()
    all_spots= [] 
    all_Rf =[] 
    
    for i in circles[0, :]: #For each of the detected circles
        centre = (i[0], i[1]) #(x,y) of the centre of the detected circles
        #y coordinate of centre is from the top of the image
        #Find distance between centre of spot from SF 
        dist_sf_spot= i[1] - positions[0] #y coord of centre minus the y coord of SF line
        #Find difference between spot and BL
        dist_bl_spot = positions[2] - dist_sf_spot #distance between SF and BL minus dist_sf_spot 
        #print(dist_bl_spot)
        #all_spots.append(dist_bl_spot)
        Rf = 0
        Rf = np.around(dist_bl_spot/positions[2], 2) #Rounding calculated Rf values to 2d.p
    
        centre = np.uint16(np.around(centre))
            
        #labelling Rf value next to spot in red
        cv2.putText(cropped_plate,str(Rf), (centre[0] + 5 , centre[1] + 25 ), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        all_Rf.append(Rf)
        for i in range(0, len(circles)):
            circle_num = circle_num + 1
            #Numering the centres on image
            cv2.putText(cropped_plate, str(circle_num),
                        (centre[0] + 5, centre[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0), 2)
            print ("\nThe distance of spot", str(circle_num), "from the pencil line is", int(dist_bl_spot),
                           "pixels.\nThe Rf values for spot" ,str(circle_num) , "is", Rf, ".")
    
    if circles is not None: #Drawing circles/centres in image
        circles = np.uint16(np.around(circles)) #All values in circles will be an integer
        for i in circles[0, :]:
            centre = (i[0], i[1])
            #To show circle centre
            #spot_positions.append(centre[1])
            cv2.circle(cropped_plate, centre, 1, (0, 0, 0), 3) # identifying centres of the circles
            #To show circle outline in pink
            #radius = i[2]
            #cv2.circle(cropped, centre, radius, (255, 0, 255), 3) 

    return all_Rf 

In [ ]:
def analyse_image(img):
    """
    This function processes the image of the plate and then detects the lines and circles of the image
    Rf values of the plate are calculated by find all the distances.
    Draw the solvent front and baseline on the cropped image of the plate and labels them 
    Returns the analysed plate
    """
    img_processed = img_Processing(img) #Processes image
    corners = getcontours(img_processed) #Use processed image to find the corners of plate
    #print (corners)

    warped_img = warp (img,corners) #Warp original image using the corners

    #Crop image after warp to simplify line detection
    cropped_plate = warped_img[15:warped_img.shape[0]-15, 15:warped_img.shape[1]-15]

    #Img_processed cropped image for to use for line detection
    processed_plate = img_Processing(cropped_plate)
    lines = all_lines(processed_plate)

    lines = lines.reshape((lines.shape[0], lines.shape[2]))
    length_threshold = processed_plate.shape[0] / 2 
    #Set threshold to categorise the detected lines into SF lines and BL lines
    positions = distances(lines, length_threshold)

    #Circle detection
    circles= findspots(cropped_plate)

    #Calculates Rf values
    all_Rf = Rf(circles,positions,cropped_plate)
    #Renaming cropped_plate to analysed plate
    analysed_plate = cropped_plate

    #Drawing the lines on TLC plate
    cv2.line(analysed_plate,(0,int(positions[0])), (processed_plate.shape[1], int(positions[0])), (0, 255, 0), 3, cv2.LINE_8 )
    cv2.line(analysed_plate,(0,int(positions[1])), (processed_plate.shape[1],int(positions[1])), (0, 255, 0), 3, cv2.LINE_8 )
    # labelling the lines
    cv2.putText(analysed_plate,"Solvent front",
                        ( 5 , (int(positions[0])-5) ) , cv2.FONT_HERSHEY_SIMPLEX , 1, (0,0,0), 2)
    cv2.putText(analysed_plate,"Pencil line",
                        ( 5 , (int(positions[1])-5) ) , cv2.FONT_HERSHEY_SIMPLEX , 1, (0,0,0), 2)
    
    return analysed_plate

In [ ]:
def save_plate (analysed_plate):
    """
    This function saves the analysed TLC plate in the same file directory as the notebook and original input image
    Saves analysed plate image in same format as input and under the same name but with "_analysed" added on
    Unless image format is .tiff or .tif, where the default is .jpg
    Code is unable to save image as .tiff/.tif
    Returns saved image 
    """
    #Checks if image is .tiff/.tif
    file_check = filename.split(".")
    file_type = file_check[1]
    
    if file_type == "tiff" or file_type == "tif": #Saves as .jpg
        new_filename = str(file_check[0]) + "_analysed."+ "jpg"
        #print(new_filename)
    else: #Saves image as same format as input image
        new_filename = str(file_check[0])+"_analysed."+ str(file_type)
    saved = cv2.imwrite(new_filename, analysed_plate)
    print("\nAnalysed plate successfully saved as",str(new_filename) , "in same file directory.")
    
    return saved

In [ ]:
def display_images(img, analysed_plate):
    """
    This function displays the original input image and the analysed plate image in the notebook using matplotlib
    The input images are converted from BGR to RGB before displaying so the saved displayed analysed plate look the same.
    """
    #Displaying original image in notebook
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.title("Original input image")
    plt.axis("off")

    #Displaying analysed_plate in notebook
    analysed_plate = cv2.cvtColor(analysed_plate, cv2.COLOR_BGR2RGB)
    plt.subplot(1,2,2)
    plt.imshow(analysed_plate)
    plt.title("Analysed plate with Rf values")
    plt.axis("off")

    show = plt.show()
    return show
    

In [ ]:
filename = input("Enter the filename of the image of the plate:(Case and number sensitive)")
#filename = "image.png"

img_resized = input_check(filename) #Checks file input

analysed_plate = analyse_image(img_resized)
saved = save_plate(analysed_plate) #Saves analysed plate
show = display_images(img_resized, analysed_plate)

#Shows orginal image and analysed plate image in pop-up windows
#cv2.imshow("Original input image",img)
#cv2.imshow("Analysed plate image", analysed_plate)
#cv2.waitKey(0)


In [ ]:
#Output checks - shows image in pop-up windows
#filename = "image1.jpg"

#img = input_check(filename)

#img_processed = img_Processing(img) 
#cv2.imshow("Original image processed",img_processed) #Shows processed image

#Change return output of getcontours function to img_Contour show img_Contour
#img_Contour = getcontours(img_processed)
#cv2.imshow("contour",img_Contour) #Shows contours and corners of the the TLC plate 

#corners = getcontours(img_processed)
#warped_img = warp (img,corners)
#cv2.imshow("Warped image of TLC plate",warped_img)# Shows warped image

#cropped_plate = warped_img[20:warped_img.shape[0]-20, 20:warped_img.shape[1]-20]
#cv2.imshow("Cropped image of TLC plate",cropped_plate) #Shows cropped image

#processed_plate = img_Processing(cropped_plate)
#cv2.imshow("Processed image of Plate for line detection", processed_plate) #Shows processed image of cropped image

#Change return output of all_lines function to copy show image
#copy = all_lines(processed_plate)
#cv2.imshow("Processed Plate,with detected lines", copy) #Shows detected lines on processed plate

#cv2.waitKey(0)
